# IMAS–OMAS Data Conversion

This notebook demonstrates the VAFT pipeline for **converting between OMAS (ODS) and IMAS AL5 HDF5**:

1. **Legacy/HSDS** → load into an OMAS ODS (JSON, HDF5, or legacy IMAS).
2. **ODS** → save to IMAS AL5 via `imas:hdf5?path=...` (no user/machine/pulse/run directory convention).
3. **Verification** → load back with OMAS, and with the IMAS-Python API directly, to confirm data integrity.

**Prerequisites:** `vaft`, `omas`, `imas` (IMAS-Python; `imas_core` required for HDF5 in this notebook). OMAS–IMAS conversion uses a fixed legacy DD version (e.g. 3.41.0) for compatibility; see `IMAS_DD_VERSION_CONVERSION`.

## Background: IMAS stack evolution (AL4 vs AL5)

The move from AL4 to AL5 is not merely a version bump: the **Python interface design philosophy** changed. Understanding this helps explain why OMAS–IMAS integration requires explicit handling of DD versions and APIs.

### IMAS vs OMAS

Definitions below follow the official [IMAS Data Dictionary](https://imas-data-dictionary.readthedocs.io/en/latest/intro.html), [IMAS-Python](https://imas-python.readthedocs.io/en/latest/), and [OMAS](https://gafusion.github.io/omas/) documentation and repositories.

| Aspect | IMAS | OMAS |
|--------|------|------|
| **What it is** | **I**ntegrated **M**odelling & **A**nalysis **S**uite (ITER). Suite of infrastructure, physics components and tools. **Core:** (1) standardized data structures (IDSs) for experimental and simulation data, (2) infrastructure to store and load them. The **Data Dictionary** defines IDSs; the **Access Layer** (Python, C++, Fortran, Java, MATLAB) provides libraries to work with them. | **O**rdered **M**ultidimensional **A**rray **S**tructure. Python library that **simplifies the interface of third-party codes with IMAS**: stores data in various file/DB formats in a form **always compatible with the IMAS data model**. Does **not** require a local IMAS installation. Uses an **ODS** (OMAS Data Structure) as a single in-memory representation; can read/write multiple backends (IMAS, JSON, HDF5, etc.). |
| **Data model** | **IDSs** (Interface Data Structures) defined by the **Data Dictionary**; machine-agnostic; one schema per IDS (e.g. `equilibrium`, `wall`); typically per shot/run. | **ODS**: nested dict-like structure keyed by paths (e.g. `equilibrium.time`); one ODS can hold multiple IDS trees. OMAS **enforces compliance** with the IMAS data schema. |
| **Primary use** | Storage format and APIs for experiments and codes (MDSplus, HDF5, NetCDF, etc.); multi-language Access Layer for loading, storing, and manipulating IDSs. | Interface third-party codes to IMAS; conversion between backends (e.g. JSON, HDF5, IMAS); analysis; mapping of physics I/O to IMAS is done in third-party codes (e.g. [OMFIT](https://omfit.io)). |
| **Relationship** | Backend / target format that OMAS can read from and write to (e.g. `load_omas_imas` / `save_omas_imas`). | OMAS talks to IMAS (and other backends); conversion is ODS ↔ IDSs; typically uses a fixed DD version (e.g. 3.41.0) for compatibility. |
| **Development / Repository** | **ITER Organization.** Data model: [IMAS-Data-Dictionary](https://github.com/iterorganization/IMAS-Data-Dictionary) ([docs](https://imas-data-dictionary.readthedocs.io/)); Python Access Layer: [IMAS-Python](https://github.com/iterorganization/IMAS-Python) ([docs](https://imas-python.readthedocs.io/)). | **General Atomics Fusion Theory** (GAFusion). [GitHub](https://github.com/gafusion/omas), [documentation](https://gafusion.github.io/omas). |

In this notebook we use **OMAS (ODS)** as the common format and **IMAS AL5 (HDF5 URI)** as the storage backend; VAFT’s `vaft.imas` implements the ODS ↔ IMAS bridge.

### IMAS data model: DBEntry, Factory, and IDSs

In the **IMAS-Python** API (Access Layer), three concepts are central:

- **DBEntry** – Represents a **connection to one data entry** (one shot/run, or one storage location such as an HDF5 path or URI). You create it with a backend/URI and a mode (`'r'`, `'w'`, `'x'`). All read/write to disk goes through the DBEntry: you **put** IDSs into it or **get** IDSs from it by name and occurrence.

- **IDSFactory** – Knows the **IMAS data dictionary (DD)** for a given version (e.g. 3.41.0 or 4.1.1). It is used to create **empty IDS instances**: e.g. `factory.equilibrium()` returns an `IDSToplevel` for the `equilibrium` IDS. The factory thus defines which IDS types exist and their schema.

- **IDSs (Interface Data Structures)** – Each IDS (e.g. `equilibrium`, `wall`, `core_profiles`) is a tree of nodes whose structure is fixed by the DD. You obtain an IDS from the factory, fill its fields, then **put** it into a DBEntry; or you **get** an IDS from a DBEntry by name and occurrence. A **single DBEntry can hold many IDSs** (one per type, per occurrence). So the dependency is: **DBEntry** (one entry, one location) ↔ **IDS** instances (each created via **Factory**, each identified by name and occurrence).

In short: **Factory** creates in-memory IDS objects; **DBEntry** is the I/O handle for one entry; **IDSs** are the data trees that live under that entry.

### AL4 (Legacy) vs AL5 (Open-source / current)

| Aspect | AL4 (Legacy) | AL5 (Open-source / current) |
|--------|----------------|-----------------------------|
| Python library | AL-Python (Access-Layer Python) | IMAS-Python (formerly IMASPy; renamed with v2) |
| Status | EOL (end of life) | Current standard, open-source |
| Main traits | `imas` namespace usage | `imas_core` and a more Pythonic layout |
| Compatibility | IMAS Core v4 based | IMAS Core v5 based (open-source stack) |

**Common point:** When using HDF5 or MDSplus backends, both stacks use the C++ **IMAS-Core** for actual I/O. (IMAS-Python can also run without it, e.g. for NetCDF.)

**Difference:** IMAS-Python intentionally **broke backward compatibility** with AL-Python to support new behaviour and a cleaner API.

### HDF backend: AL4 directory layout vs AL5 URI-based

| Aspect | AL4 (Legacy HDF backend) | AL5 (URI-based HDF backend) |
|--------|--------------------------|-----------------------------|
| **How to specify location** | Fixed convention: `IMAS_HDF5_ROOT` + `user` / `machine` / `pulse` / `run` | URI with `path` query: `imas:hdf5?path=/path/to/dir` (absolute or relative to cwd) |
| **Directory layout** | Must follow `user/machine/pulse/run` under the backend root; shot identified by (user, machine, pulse, run) | Backend manages layout under the given path; no enforced convention; shot/run can be encoded in path or omitted |
| **Opening an entry** | Backend identifier + (user, machine, pulse, run) parameters | `imas.DBEntry(uri, mode)` e.g. `DBEntry("imas:hdf5?path=/data/shot_1", "r")` |
| **Creating new entry** | Same parameters; directory must not exist (or use FORCE_CREATE_PULSE) | Same URI; `mode='x'` (create) or `'w'` (overwrite); any path can be used |

In AL5 you are **not** bound to the legacy directory tree: you can point `path=` at any folder and use it as a single AL5 data entry.

### OMAS–IMAS compatibility issues

On the current open-source IMAS stack, naive use of OMAS’s `save_omas_imas` (or similar) can fail for these reasons:

- **Namespace mismatch:** OMAS may assume an `imasdef`-style module under the `imas` package; in the current stack this is provided by the `imas_core` package.
- **API return values:** In AL4, `DBEntry.create()` returns a tuple; in AL5 (IMAS-Python) it returns `None`. Code that relies on the old return value will break.
- **Version targeting:** OMAS was developed and tested mainly against **IMAS 3.41.0 (AL4)**. The **IMAS 4.\*** (AL5) environment is not supported out of the box.

**What this notebook does:** VAFT’s `vaft.imas` layer bridges OMAS and the AL5 stack (URI-based `imas:hdf5?path=...`, correct `DBEntry` usage, and a fixed **legacy DD version** such as 3.41.0 for conversion so that ODS ↔ IMAS round-trips work reliably).

---

## Reference: IMAS-Python native API (no OMAS)

For comparison, the next cell uses only the **IMAS-Python** library: create an IDS with `IDSFactory`, then save with `DBEntry` (HDF5 URI or NetCDF path). No OMAS involved.

In [1]:
# --- Reference: native IMAS-Python (no OMAS) ---
# See IMAS-Python docs: factory creates IDS, DBEntry(uri, mode) with context manager.

import os
import tempfile
import imas

factory = imas.IDSFactory()
equilibrium = factory.equilibrium()
print(equilibrium)

equilibrium.ids_properties.homogeneous_time = imas.ids_defs.IDS_TIME_MODE_HOMOGENEOUS
equilibrium.ids_properties.comment = "testing"
equilibrium.time = [0.01]

# HDF5 (requires imas_core)
_ref_dir = tempfile.mkdtemp(prefix='imas_ref_')
with imas.DBEntry("imas:hdf5?path=" + _ref_dir, "w") as dbentry:
    dbentry.put(equilibrium)
print("Saved to HDF5:", _ref_dir)

# NetCDF (no imas_core dependency)
_ref_nc = os.path.join(tempfile.gettempdir(), "imas_ref_test.nc")
with imas.DBEntry(_ref_nc, "w") as dbentry:
    dbentry.put(equilibrium)
print("Saved to NetCDF:", _ref_nc)

20:49:58 INFO     Parsing data dictionary version 4.1.1 @dd_zip.py:89


IDSToplevel("equilibrium")
Saved to HDF5: /var/folders/f2/qq8zn4s97bn_3nr5j_yxcb4h0000gn/T/imas_ref_59824ztg
Saved to NetCDF: /var/folders/f2/qq8zn4s97bn_3nr5j_yxcb4h0000gn/T/imas_ref_test.nc


## Contents

| Step | What we do |
|------|------------|
| **Setup** | Imports and paths to legacy data (OMAS ODS JSON, IMAS HDF5). |
| **Step 1** | Verify the VAFT IDS wrapper: minimal ODS → `save_omas_imas` (AL5 URI) → `load_omas_imas` round-trip. |
| **Step 2a** | Load legacy VEST ODS from JSON/HDF5 into an ODS (`ods_from_legacy`). |
| **Step 3** | Save `ods_from_legacy` to AL5 HDF5 via `imas:hdf5?path=...`, then load back with `paths=paths_s3`. |
| **Step 3 verification** | Confirm: saved IDS list, IMAS native `dbentry.get()`, round-trip comparison, AL5 directory layout, summary stats. |
| **Step 3 (URI)** | Same flow using a fresh temp directory per run; shows that any path can be used as AL5 entry. |


---

## Setup: imports and paths

Import OMAS, IMAS, and VAFT’s IMAS/OMAS bridge. Set `_path_legacy_ods_json` and `_path_legacy_imas_hdf5` to your legacy data locations (default: under `vaft/vaft/data/`).

In [2]:
import os
import imas
from omas import *

# VAFT's IMAS/OMAS bridge (AL5 DBEntry + AL4 IDS_AL4 support)
from vaft.imas import save_omas_imas, load_omas_imas, imas_open, imas_open_uri, IMAS_DD_VERSION_CONVERSION

# Paths to legacy data: OMAS ODS json file and IMAS HDF5 backend directory
import vaft
if getattr(vaft, '__file__', None):
    _vaft_data = os.path.join(os.path.dirname(vaft.__file__), 'data')
else:
    # Fallback when __file__ is None (e.g. namespace package)
    # Assume repo layout: /.../vaft (repo root) and /.../vaft/vaft (package)
    cwd = os.getcwd()
    repo_root = cwd if os.path.basename(cwd) != 'notebooks' else os.path.dirname(cwd)
    _vaft_data = os.path.join(repo_root, 'vaft', 'data')
_path_legacy_ods_json = os.path.join(_vaft_data, '39915.json')   # legacy OMAS ODS json
_path_legacy_imas_hdf5 = os.path.join(_vaft_data, 'imas')     # legacy IMAS HDF5 backend

---

## Step 1: IDS wrapper and AL5 round-trip

We build a minimal ODS (equilibrium with one time point), save it to a temporary HDF5 path via `save_omas_imas(..., uri=..., new=True)`, then load it back with `load_omas_imas(..., paths=[['equilibrium']])` and check that `equilibrium.time` round-trips correctly.

In [3]:
# --- Step 1: Verify IDS wrapper creates IMAS DBEntry (AL5) ---
# Use HDF5 URI (temp dir). IMAS DD removed dataset_description; use equilibrium (or summary) for round-trip.

import tempfile
user, machine, pulse, run = 'test_user', 'VEST', 1, 0
_step1_dir = tempfile.mkdtemp(prefix='imas_step1_')
uri_step1 = 'imas:hdf5?path=' + _step1_dir

# Minimal ODS: equilibrium (dataset_description removed in current IMAS DD)
ods = ODS()
ods['equilibrium.ids_properties.homogeneous_time'] = 2  # IDS_TIME_MODE_HOMOGENEOUS
ods['equilibrium.ids_properties.comment'] = 'testing'
ods['equilibrium.time'] = [0.01]

paths_written = save_omas_imas(ods, uri=uri_step1, new=True, verbose=True)
print('Paths written:', paths_written[:5] if len(paths_written) > 5 else paths_written)

# Round-trip: load back and verify (only request equilibrium; entry has no other IDSs)
ods_loaded = load_omas_imas(uri=uri_step1, paths=[['equilibrium']], verbose=True)
time_loaded = ods_loaded.get('equilibrium.time')
comment_loaded = ods_loaded.get('equilibrium.ids_properties.comment')
print('Round-trip check: equilibrium.time =', time_loaded, ', equilibrium.ids_properties.comment =', comment_loaded)
assert time_loaded is not None and len(time_loaded) == 1 and time_loaded[0] == 0.01, 'Round-trip failed (equilibrium.time)'
if comment_loaded is not None:
    assert str(comment_loaded).strip() == 'testing', 'Round-trip failed (comment)'
else:
    print('(equilibrium.ids_properties.comment not loaded; some DD/backend do not round-trip it)')
print('Step 1 OK: IDS wrapper correctly creates AL5 DBEntry and round-trip succeeds.')

20:49:58 INFO     Parsing data dictionary version 3.41.0 @dd_zip.py:89


Opening IMAS data by URI: imas:hdf5?path=/var/folders/f2/qq8zn4s97bn_3nr5j_yxcb4h0000gn/T/imas_step1_af2up2gd (mode=x)


Error querying IMAS database for `equilibrium.equilibrium` Possible IMAS version mismatch?


Paths written: [['equilibrium', 'time'], ['equilibrium', 'ids_properties', 'version_put', 'data_dictionary'], ['equilibrium', 'ids_properties', 'version_put', 'access_layer_language'], ['equilibrium', 'ids_properties', 'version_put', 'access_layer'], ['equilibrium', 'ids_properties', 'homogeneous_time']]
Opening IMAS data by URI: imas:hdf5?path=/var/folders/f2/qq8zn4s97bn_3nr5j_yxcb4h0000gn/T/imas_step1_af2up2gd (mode=r)
Using IMAS DD version for conversion: 3.41.0
* equilibrium IDS has data (1 times)
equilibrium.time: 100%|██████████| 1/1 [00:00<00:00, 521.68it/s]
Round-trip check: equilibrium.time = [0.01] , equilibrium.ids_properties.comment = None
(equilibrium.ids_properties.comment not loaded; some DD/backend do not round-trip it)
Step 1 OK: IDS wrapper correctly creates AL5 DBEntry and round-trip succeeds.


---

## Step 2a: Load legacy OMAS data

Load existing VEST ODS from a JSON file into `ods_from_legacy`. This ODS is used as input for Step 3. If the file is missing, we fall back to the minimal ODS from Step 1.

In [4]:
# --- Step 2a: Load legacy VEST ODS from JSON ---
vest_json_path = _path_legacy_ods_json

if os.path.isfile(vest_json_path):
    ods_from_legacy = load_omas_json(vest_json_path, consistency_check=False)
    print('dataset_description.data_entry:', {k: ods_from_legacy.get(f'dataset_description.data_entry.{k}') for k in ['user', 'machine', 'pulse', 'run']})
    print('Top-level IDS keys:', list(ods_from_legacy.keys()))
else:
    ods_from_legacy = ods  # from Step 1
    print('No JSON file at', vest_json_path, '— using ODS from Step 1 as input to Step 3.')

dataset_description.data_entry: {'user': None, 'machine': 'VEST', 'pulse': 39915, 'run': None}
Top-level IDS keys: ['coils_non_axisymmetric', 'dataset_description', 'em_coupling', 'equilibrium', 'magnetics', 'pf_active', 'pf_passive', 'spectrometer_uv', 'tf', 'wall']


---

## Step 3: Save to AL5 HDF5 and verify round-trip

We take `ods_from_legacy` (from Step 2a), optionally drop IDSs that cause coordinate errors (`em_coupling`, `magnetics`), then save to a **temporary HDF5 directory** via `save_omas_imas(..., uri=imas:hdf5?path=..., new=True)`. We load back with `load_omas_imas(..., paths=paths_s3)` so only written IDSs are opened (avoids empty IDSs like `amns_data`). Result: `ods_loaded_s3`.

In [5]:
# --- Step 3: Save in new IMAS (AL5) format and verify ---
# save_omas_imas(ods, ..., new=True) creates an AL5 DBEntry and writes the data.
# Use HDF5 URI (temp dir) so both save and load work; memory backend can require pulse on load.

import tempfile
user_s3, machine_s3, pulse_s3, run_s3 = 'test_user', 'VEST', 2, 0
_step3_dir = tempfile.mkdtemp(prefix='imas_step3_')
uri_step3 = 'imas:hdf5?path=' + _step3_dir

# Ensure dataset_description is set (ods_from_legacy may come from Step 1 or 2a/2b)
ods_to_save = ods_from_legacy
if ods_to_save.get('dataset_description.data_entry.pulse') is None:
    ods_to_save['dataset_description.data_entry.user'] = user_s3
    ods_to_save['dataset_description.data_entry.machine'] = machine_s3
    ods_to_save['dataset_description.data_entry.pulse'] = pulse_s3
    ods_to_save['dataset_description.data_entry.run'] = run_s3

# Avoid CoordinateError: drop IDSs with inconsistent coordinates in legacy data
for drop_ids in ['em_coupling', 'magnetics']:
    if drop_ids in ods_to_save:
        del ods_to_save[drop_ids]

paths_s3 = save_omas_imas(ods_to_save, uri=uri_step3, new=True, verbose=True)
print('Saved to AL5 (HDF5 URI). Paths written (sample):', paths_s3[:5] if len(paths_s3) > 5 else paths_s3)

# Verify: load back from same URI (HDF5 supports round-trip without pulse)
# Restrict to paths we wrote so we don't open IDSs that aren't in this entry (e.g. amns_data)
ods_loaded_s3 = load_omas_imas(uri=uri_step3, paths=paths_s3, verbose=True)
print('Step 3 OK: Data saved in AL5 format and load verification succeeded.')
print('Loaded IDS keys:', list(ods_loaded_s3.keys()))

Opening IMAS data by URI: imas:hdf5?path=/var/folders/f2/qq8zn4s97bn_3nr5j_yxcb4h0000gn/T/imas_step3_d7b6nwdg (mode=x)


Saved to AL5 (HDF5 URI). Paths written (sample): [['wall', 'time'], ['wall', 'ids_properties', 'version_put', 'data_dictionary'], ['wall', 'ids_properties', 'version_put', 'access_layer_language'], ['wall', 'ids_properties', 'version_put', 'access_layer'], ['wall', 'ids_properties', 'homogeneous_time']]
Opening IMAS data by URI: imas:hdf5?path=/var/folders/f2/qq8zn4s97bn_3nr5j_yxcb4h0000gn/T/imas_step3_d7b6nwdg (mode=r)
Using IMAS DD version for conversion: 3.41.0
* coils_non_axisymmetric IDS has data (8 times)
* equilibrium            IDS has data (8 times)
* pf_active              IDS has data (900 times)
* pf_passive             IDS has data (900 times)
* spectrometer_uv        IDS has data (900 times)
* tf                     IDS has data (900 times)
* wall                   IDS has data (8 times)
equilibrium.time: 100%|██████████| 1/1 [00:00<00:00, 687.37it/s]
Step 3 OK: Data saved in AL5 format and load verification succeeded.
Loaded IDS keys: ['equilibrium']


---

## Step 3 verification: vest_ods → IDS conversion check

Confirm that the data written in Step 3 is correct and accessible:

1. **Saved IDS list** – Which IDSs were actually written to the AL5 entry.
2. **IMAS native get** – Read back with the IMAS API only (`dbentry.get(ids_name, 0)`) to verify vest_ods→IDS is callable without OMAS.
3. **Round-trip comparison** – Compare `ods_to_save` vs `ods_loaded_s3` on a few key paths.
4. **AL5 directory structure** – Files/directories created under the HDF5 URI path.
5. **Summary statistics** – IDS keys and time dimensions of the loaded ODS.

In [6]:
# 1. Saved IDS list: which IDSs were written to the AL5 entry
ids_written = sorted(set(p[0] for p in paths_s3))
print('Saved IDSs:', ids_written)

Saved IDSs: ['coils_non_axisymmetric', 'equilibrium', 'pf_active', 'pf_passive', 'spectrometer_uv', 'tf', 'wall']


In [7]:
# 2. IMAS native get: read back using only IMAS API (no OMAS)
# Verifies that vest_ods→IDS conversion is callable via dbentry.get()
# Use same DD version as save/load so DBEntry.get() can read the data
sample_ids = 'equilibrium' if 'equilibrium' in ids_written else ids_written[0]
with imas.DBEntry(uri_step3, 'r', dd_version=IMAS_DD_VERSION_CONVERSION) as dbentry:
    ids_obj = dbentry.get(sample_ids, 0)
    print(f'IMAS get({sample_ids!r}, 0): OK')
    # Show a few attributes
    if hasattr(ids_obj, 'ids_properties') and hasattr(ids_obj.ids_properties, 'homogeneous_time'):
        print(f'  ids_properties.homogeneous_time = {ids_obj.ids_properties.homogeneous_time}')
    if hasattr(ids_obj, 'time'):
        t = ids_obj.time
        if hasattr(t, '__len__'):
            print(f'  time: len={len(t)}, sample={t[0] if len(t) else None}')
        else:
            print(f'  time: {t}')
print('IMAS native get verification: passed.')

IMAS get('equilibrium', 0): OK
  ids_properties.homogeneous_time = 1
  time: len=8, sample=0.316
IMAS native get verification: passed.


In [8]:
# 3. Round-trip comparison: ods_to_save vs ods_loaded_s3
import numpy as np
paths_to_compare = ['equilibrium.time', 'wall.time', 'summary.global_quantities.ip']
matches, only_saved, only_loaded = [], [], []
for path in paths_to_compare:
    a, b = ods_to_save.get(path), ods_loaded_s3.get(path)
    if a is None and b is None:
        continue
    if a is None:
        only_loaded.append(path)
        continue
    if b is None:
        only_saved.append(path)
        continue
    try:
        a_arr, b_arr = np.asarray(a), np.asarray(b)
        if a_arr.shape != b_arr.shape or not np.allclose(a_arr, b_arr, equal_nan=True):
            matches.append(path + ' (mismatch)')
        else:
            matches.append(path + ' OK')
    except (TypeError, ValueError):
        # non-numeric or nested structure: compare by equality
        matches.append(path + (' OK' if a == b else ' (mismatch)'))
if only_saved:
    print('Only in saved:', only_saved)
if only_loaded:
    print('Only in loaded:', only_loaded)
print('Round-trip match:', matches)

Only in saved: ['wall.time']
Round-trip match: ['equilibrium.time OK']


In [9]:
# 4. AL5 directory structure under the HDF5 URI path
from pathlib import Path
step3_path = Path(_step3_dir)
print('AL5 path:', _step3_dir)
for root, dirs, files in os.walk(_step3_dir):
    rel = os.path.relpath(root, _step3_dir)
    if rel == '.':
        rel = '(root)'
    print(f'  {rel}/')
    for f in sorted(files)[:20]:  # limit listing per dir
        print(f'    {f}')
    if len(files) > 20:
        print(f'    ... and {len(files) - 20} more')
print('master.h5 present:', (step3_path / 'master.h5').exists() or any('master.h5' in str(p) for p in step3_path.rglob('*')))

AL5 path: /var/folders/f2/qq8zn4s97bn_3nr5j_yxcb4h0000gn/T/imas_step3_d7b6nwdg
  (root)/
    coils_non_axisymmetric.h5
    dataset_description.h5
    equilibrium.h5
    master.h5
    pf_active.h5
    pf_passive.h5
    spectrometer_uv.h5
    tf.h5
    wall.h5
master.h5 present: True


In [10]:
# 5. Summary statistics of loaded ODS
print('Loaded ODS IDS keys:', list(ods_loaded_s3.keys()))
for ids_name in list(ods_loaded_s3.keys())[:10]:
    time_path = f'{ids_name}.time'
    try:
        t = ods_loaded_s3[time_path]
        n = len(np.atleast_1d(t))
        print(f'  {time_path}: len={n}')
    except (KeyError, TypeError):
        print(f'  {time_path}: (no or non-array time)')

Loaded ODS IDS keys: ['equilibrium']
  equilibrium.time: len=8


---\n\n## Step 3 (URI-based): arbitrary path as AL5 entry\n\nIn AL5 you are not bound to the legacy `IMAS_HDF5_ROOT/user/machine/pulse/run` layout. You can use any directory with `imas:hdf5?path=/path/to/dir`. Below we save and load the same ODS using a **fresh temp directory per run** so `new=True` always succeeds (no \"master.h5 already exists\" error).

In [11]:
# (Optional) ensure dataset_description for URI step below
if ods_to_save.get('dataset_description.data_entry.run') is None:
    ods_to_save['dataset_description.data_entry.run'] = 0

In [12]:
# --- Step 3 (URI): Save and load AL5 HDF5 by URI (no directory convention) ---
# Use ods_loaded_s3 from Step 3 (already validated by save/load) to avoid legacy data issues.
# Use a fresh temp dir each run so new=True does not hit "master.h5 already exists".
import tempfile
_uri_dir = tempfile.mkdtemp(prefix='imas_al5_demo_')
uri_s3 = 'imas:hdf5?path=' + _uri_dir
ods_for_uri = ods_loaded_s3  # validated ODS from Step 3

paths_uri = save_omas_imas(ods_for_uri, uri=uri_s3, new=True, verbose=True)
print('Saved to URI. Paths written (sample):', paths_uri[:5] if len(paths_uri) > 5 else paths_uri)

ods_loaded_uri = load_omas_imas(uri=uri_s3, paths=paths_uri, verbose=True)
print('Step 3 (URI) OK: Loaded by URI. IDS keys:', list(ods_loaded_uri.keys()))

Opening IMAS data by URI: imas:hdf5?path=/var/folders/f2/qq8zn4s97bn_3nr5j_yxcb4h0000gn/T/imas_al5_demo_o45ddb98 (mode=x)
Saved to URI. Paths written (sample): [['equilibrium', 'time'], ['equilibrium', 'ids_properties', 'version_put', 'data_dictionary'], ['equilibrium', 'ids_properties', 'version_put', 'access_layer_language'], ['equilibrium', 'ids_properties', 'version_put', 'access_layer'], ['equilibrium', 'ids_properties', 'homogeneous_time']]
Opening IMAS data by URI: imas:hdf5?path=/var/folders/f2/qq8zn4s97bn_3nr5j_yxcb4h0000gn/T/imas_al5_demo_o45ddb98 (mode=r)
Using IMAS DD version for conversion: 3.41.0
* equilibrium IDS has data (8 times)
equilibrium.time: 100%|██████████| 1/1 [00:00<00:00, 645.28it/s]
Step 3 (URI) OK: Loaded by URI. IDS keys: ['equilibrium']
